### NLP Limpieza inicial de texto

- Instalaciones 

In [23]:
# Si necesitas spaCy para etapas posteriores:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install dateparser




[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
      --------------------------------------- 0.2/12.9 MB 2.6 MB/s eta 0:00:05
     ---- ----------------------------------- 1.4/12.9 MB 12.8 MB/s eta 0:00:01
     ----------- ---------------------------- 3.7/12.9 MB 23.4 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/12.9 MB 39.6 MB/s eta 0:00:01
     --------------------------------------  12.9/12.9 MB 93.0 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 72.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


- Importaciones

In [24]:
import re
import string
import json
import dateparser
import spacy
import pandas as pd

- Definición de la función de limpieza

In [25]:
import re
import string
import json
import dateparser

# --- 1. Limpieza inicial del texto ---
def limpiar_texto_comercial(texto: str) -> str:
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)  # URLs
    texto = re.sub(r'@\w+', '', texto)                   # Handles
    texto = re.sub(r'[\[\](){}]', '', texto)             # Paréntesis y corchetes
    permitidos = '!?\$%/'
    eliminar = ''.join(c for c in string.punctuation if c not in permitidos)
    texto = texto.translate(str.maketrans('', '', eliminar))  # Puntuación no permitida
    texto = re.sub(r'\s+', ' ', texto)                  # Espacios múltiples y saltos
    return texto.strip()

# --- 2. Normalización de números, unidades y formatos clave ---
def normalizar_numeros_unidades(texto: str) -> str:
    def reemplazar_fechas(match):
        fecha_original = match.group(0)
        fecha = dateparser.parse(fecha_original)
        return fecha.strftime('%Y-%m-%d') if fecha else fecha_original

    patron_fechas = r'\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{1,2} de \w+ de \d{4}|\w+ \d{1,2}, \d{4})\b'
    texto = re.sub(patron_fechas, reemplazar_fechas, texto)

    texto = re.sub(r'\b(\d+)\s?(x|kg|ml|unidades)\b', r'\1_\2', texto, flags=re.IGNORECASE)
    texto = re.sub(r'(?<!\d)\$ ?(\d+(?:[\.,]\d{1,2})?)', r'<USD>\1', texto)
    texto = re.sub(r'(\d+(?:[\.,]\d{1,2})?)\$', r'<USD>\1', texto)
    texto = re.sub(r'\b(?<![%\d])\d{2,}(?![%\d])\b', '<NUM>', texto)

    return texto

# --- 3. Función combinada final ---
def procesar_texto_comercial(texto: str) -> str:
    texto = limpiar_texto_comercial(texto)
    texto = normalizar_numeros_unidades(texto)
    return texto

# 6. Leer y procesar JSON y devolver ambos textos
def procesar_json_ruta(ruta_archivo: str):
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        data = json.load(f)

    resultados = []
    for i, entrada in enumerate(data):
        texto_original = entrada.get("input", "")
        texto_procesado = procesar_texto_comercial(texto_original)
        resultado = {
            "original": texto_original,
            "procesado": texto_procesado
        }
        print(f"\n📝 Texto {i+1}")
        print(f"🔹 Original:   {resultado['original']}")
        print(f"🔹 Procesado: {resultado['procesado']}")
        resultados.append(resultado)

    return resultados

- Prueba

In [26]:
# 7. Ejecutar
resultados = procesar_json_ruta("validation_samples.json")


📝 Texto 1
🔹 Original:   🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.
🔹 Procesado: 🔥¡OFERTA! Compre 2_x zapatos Nike a <USD><NUM> antes <USD><NUM> 👟 ¡Válido hasta el <NUM>-<NUM>-<NUM>! Visita Atención ¿Envío gratis? 😃 ModaDeportiva2023

📝 Texto 2
🔹 Original:   I love my new iPhone 12! 😍 Battery life is amazing. Bought it at 799€ from https://apple.com. #TechReview
🔹 Procesado: I love my new iPhone <NUM>! 😍 Battery life is amazing Bought it at <NUM>€ from TechReview

📝 Texto 3
🔹 Original:   @Juan gracias por la promo! Compré 3kg de café a $15.00. Llegó en 24h 🕒. Calidad 10/10 👍.
🔹 Procesado: gracias por la promo! Compré 3_kg de café a <USD><NUM> Llegó en 24h 🕒 Calidad <NUM>/<NUM> 👍

📝 Texto 4
🔹 Original:   Great deal: 5x shirts on sale for $49.99 (was $80) – limited time until 12/25/2023! Visit our store now.
🔹 Procesado: Great deal 5_x shirts on sa